In [1]:
import openmc.data
import pandas as pd
import sqlite3
import numpy as np
import os
from helper_functions import get_A, get_element, get_z, extract_XS_openMC_ace
import matplotlib.pyplot as plt

In [2]:
# DATA PATH CONFIGURATION

# The datasets used in this work are all too large to be included with this repo. 

# openMC's neutron hdf5 data is used by default. 
# This can be replaced with your own ACE files if desired, but you will need to change other functions.
endf7_1_directory = "/global/scratch/users/co_nuclear/endf71x/<symbol>/<ZAID>.710nc"

endf8_directory = "/global/home/groups/co_nuclear/serpent/xsdata/endf8/Lib80x/<symbol>/<ZAID>.800nc"

# x4sqlite1.db location.
# Find it here: https://www-nds.iaea.org/cdroms/
x4sqlite_directory = "sources/x4sqlite1.db"


output_directory = "output/"

In [3]:
# X4 data types 
data_types = {'Reaction': str, 'Projectile': str, 'MT': np.int16, 'Target': str,
              'En': np.float64, 'dEn': np.float64, 'Sig': np.float64, 'dSig': np.float64,
              'fullCode': str, 'YearRef1': np.int16, 'Author1Ini': str, 'Author1': str,
              'DatasetID': str, 'Subent': str}

In [4]:
# Extract SQL data into simple DataFrame
con = sqlite3.connect(x4sqlite_directory)
query = "SELECT Reaction, Projectile, En, dEn, Sig, dSig, MT, DatasetID, Subent, YearRef1, Author1Ini, Author1, Target, fullCode from sig1"
exfor_df = pd.read_sql_query(query, con, dtype=data_types)
print(exfor_df.head())
con.close()

  Reaction Projectile           En         dEn     Sig    dSig  MT DatasetID  \
0    N,TOT          n  247363000.0  15270500.0  0.0202  0.0011   1  10365005   
1    N,TOT          n  278540000.0  15905900.0  0.0221  0.0007   1  10365005   
2    N,TOT          n  310939000.0  16493300.0  0.0233  0.0008   1  10365005   
3    N,TOT          n  344468000.0  17036100.0  0.0249  0.0005   1  10365005   
4    N,TOT          n  379042000.0  17537500.0  0.0266  0.0003   1  10365005   

     Subent  YearRef1 Author1Ini Author1 Target            fullCode  
0  10365005      1973       T.J.  Devlin   NN-1  0-NN-1(N,TOT),,SIG  
1  10365005      1973       T.J.  Devlin   NN-1  0-NN-1(N,TOT),,SIG  
2  10365005      1973       T.J.  Devlin   NN-1  0-NN-1(N,TOT),,SIG  
3  10365005      1973       T.J.  Devlin   NN-1  0-NN-1(N,TOT),,SIG  
4  10365005      1973       T.J.  Devlin   NN-1  0-NN-1(N,TOT),,SIG  


In [5]:
# Reformat and rename EXFOR data
exfor_df['A'] = exfor_df[['Target']].applymap(get_A)
exfor_df['Element'] = exfor_df[['Target']].applymap(get_element)
exfor_df['Z'] = exfor_df[['Element']].applymap(get_z)
exfor_df['Author'] = exfor_df['Author1Ini'] + exfor_df['Author1']
exfor_df.rename(columns={"En": "Energy", "dEn": "dEnergy", "Subent": "EXFOR_Entry", "DatasetID": "Dataset_Number",
                         "YearRef1": "Year", "Sig": "Data", "dSig": "dData"}, inplace=True)
display(exfor_df.head())



/tmp/ipykernel_495991/1121761506.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  exfor_df['A'] = exfor_df[['Target']].applymap(get_A)
/tmp/ipykernel_495991/1121761506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  exfor_df['Element'] = exfor_df[['Target']].applymap(get_element)
/tmp/ipykernel_495991/1121761506.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  exfor_df['Z'] = exfor_df[['Element']].applymap(get_z)


,Reaction,Projectile,Energy,dEnergy,Data,dData,MT,Dataset_Number,EXFOR_Entry,Year,Author1Ini,Author1,Target,fullCode,A,Element,Z,Author
0,"N,TOT",n,247363000.0,15270500.0,0.0202,0.0011,1,10365005,10365005,1973,T.J.,Devlin,NN-1,"0-NN-1(N,TOT),,SIG",1,NN,0,T.J.Devlin
1,"N,TOT",n,278540000.0,15905900.0,0.0221,0.0007,1,10365005,10365005,1973,T.J.,Devlin,NN-1,"0-NN-1(N,TOT),,SIG",1,NN,0,T.J.Devlin
2,"N,TOT",n,310939000.0,16493300.0,0.0233,0.0008,1,10365005,10365005,1973,T.J.,Devlin,NN-1,"0-NN-1(N,TOT),,SIG",1,NN,0,T.J.Devlin
3,"N,TOT",n,344468000.0,17036100.0,0.0249,0.0005,1,10365005,10365005,1973,T.J.,Devlin,NN-1,"0-NN-1(N,TOT),,SIG",1,NN,0,T.J.Devlin
4,"N,TOT",n,379042000.0,17537500.0,0.0266,0.0003,1,10365005,10365005,1973,T.J.,Devlin,NN-1,"0-NN-1(N,TOT),,SIG",1,NN,0,T.J.Devlin


In [6]:
relative_unc = exfor_df["dData"]/exfor_df["Data"]
relative_unc = relative_unc.replace(np.inf, np.nan)
relative_unc = relative_unc.dropna()
Q90_ERROR = np.quantile(relative_unc, 0.90)
print(f"90th Quantile Relative Uncertainty: {round(Q90_ERROR*100, 3)}%")

90th Quantile Relative Uncertainty: 36.145%


In [7]:
# Get list of unique isotopes and write it for NuGrade
all_isotopes = exfor_df.drop_duplicates(subset=['Z', 'A'])[["Z", "A", "Element"]]
reaction_summary_list = []
neutron_reaction_channels = {"N,TOT":1, "N,EL":2, "N,INL":4, "N,G":102}
proton_reaction_channels = {"P,EL":2, "P,INL":103, "P,G":102}

data_file_columns = ['Energy', 'dEnergy', 'Data', 'dData', 'EXFOR_Entry',
                     'Year', 'Author', 'Dataset_Number', 'MT']

In [8]:
def compute_channel_data(eval_path, evaluation, channel_data, mt, energy_values, output_directory, quantile=0.9, fallback_error=Q90_ERROR):
    if not os.path.exists(output_directory + "evals/" + evaluation):
        os.mkdir(output_directory + "evals/" + evaluation)
    found_eval = False
    # Check if evaluation exists, and then check if evaluation has this MT
    if os.path.isfile(eval_path):
        try:
            interp_xs, grid_energy, grid_xs = extract_XS_openMC_ace(eval_path, mt, energy_values)
            found_eval = True
        except KeyError:
            found_eval = False
        except AssertionError:
            found_eval = False
            print(f"AssertionError for {eval_path}")
    else:
        found_eval = False

    # fill missing values for uncertainty
    channel_data_imputed = channel_data.copy()

    # Use 90% percentile within this nuclide/reaction's data (max 1000% error to handle likely input errors)
    if len(channel_data.dropna()) > 0:
        relative_unc = channel_data["dData"]/channel_data["Data"]
        relative_unc = relative_unc.replace(np.inf, np.nan)
        relative_unc = relative_unc.dropna()
        assumed_error_fraction = np.min([np.quantile(relative_unc, 0.90), 10])
    # If no data in this nuclide/reaction's data has any uncertainty, fall back a global assumption
    else:
        assumed_error_fraction = fallback_error
    channel_data_imputed["dData"] = channel_data_imputed["dData"].fillna(channel_data_imputed["Data"]*assumed_error_fraction)
    channel_data["dData_assumed"] = channel_data_imputed["dData"]

    # If evaluation exists, compute relative uncertainty and chi squared relative to it
    if found_eval:
        channel_data[evaluation] = interp_xs

        channel_data[evaluation + '_chi_squared'] = ((channel_data_imputed['Data'] - channel_data[evaluation]) ** 2) / \
                                                    channel_data_imputed['dData']
        channel_data[evaluation + '_relative_error'] = (channel_data['Data'] - channel_data[evaluation]) / channel_data[
            evaluation] * 100
        channel_data[evaluation + '_relative_error'] = channel_data[evaluation + '_relative_error'].replace(np.inf, np.nan)
        
        file_name = output_directory + "evals/" + evaluation+"/"+ f"{proj}_{Z}_{A}_{reaction}_grid_data.csv"
        
        grid_df = pd.DataFrame({"grid_energy(eV)":grid_energy, "grid_xs(b)":grid_xs})
        grid_df.to_csv(file_name, na_rep='nan')

    # If not, set to nan indicating there isn't one
    else:
        f"Reading {evaluation} failed for {symbol}{reaction}."
        channel_data[evaluation] = np.nan
        channel_data[evaluation + "_relative_error"] = np.nan
        channel_data[evaluation + "_chi_squared"] = np.nan
        print(f"{eval_path} not found. Evaluation does not exist?")
    return channel_data, found_eval

In [9]:
# Iterate through unique isotopes
for index, row in all_isotopes.iterrows():
    Z = row.iloc[0]
    A = row.iloc[1]
    symbol = row.iloc[2]
    symbol_caps = symbol[0].upper() + symbol[1:]
    if Z < 1 or A < 1:
        continue
    ZAID = str(Z) + str(A).zfill(3)



    
    for reaction in neutron_reaction_channels.keys():
        print(reaction)
        mt = neutron_reaction_channels[reaction]
        proj = reaction[0].lower()
        channel_data = exfor_df.loc[(exfor_df['Z'] == Z)
                                    & (exfor_df['A'] == A)
                                    & (exfor_df['Reaction'] == reaction)]
        channel_data = channel_data[data_file_columns]
        if len(channel_data.index) == 0:
            continue
        else:
            channel_summary = [Z,A,symbol,proj,mt, reaction]
        channel_data = channel_data.sort_values(by=['Energy'])
        energy_values = channel_data['Energy'].to_numpy()

        
        eval_path = endf7_1_directory.replace("<symbol>", symbol_caps).replace("<ZAID>",ZAID)
        evaluation = "endf7-1"
        channel_data, eval_exists = compute_channel_data(eval_path, evaluation, channel_data, mt, energy_values, output_directory)
        channel_summary += [eval_exists]
        
        eval_path = endf8_directory.replace("<symbol>", symbol_caps).replace("<ZAID>",ZAID)
        evaluation = "endf8"
        channel_data, eval_exists = compute_channel_data(eval_path, evaluation, channel_data, mt, energy_values, output_directory)
        channel_summary += [eval_exists]
        
        file_name = output_directory + f"{proj}_{Z}_{A}_{reaction}.csv"
        channel_data.to_csv(file_name, na_rep='nan')
        reaction_summary_list += [channel_summary]



N,TOT


/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/energy_distribution.py:1244: UserWarning: Interpolation scheme for continuous tabular distribution is not histogram or linear-linear.
  warn("Interpolation scheme for continuous tabular distribution "
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/energy_distribution.py:1244: UserWarning: Interpolation scheme for continuous tabular distribution is not histogram or linear-linear.
  warn("Interpolation scheme for continuous tabular distribution "


N,EL


/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/energy_distribution.py:1244: UserWarning: Interpolation scheme for continuous tabular distribution is not histogram or linear-linear.
  warn("Interpolation scheme for continuous tabular distribution "
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/energy_distribution.py:1244: UserWarning: Interpolation scheme for continuous tabular distribution is not histogram or linear-linear.
  warn("Interpolation scheme for continuous tabular distribution "


N,INL


...

N,EL


/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=28 but no cross section is given.
  warn('Photon production is present for MT={} but no '
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=32 but no cross section is given.
  warn('Photon production is present for MT={} but no '
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=28 but no cross section is given.
  warn('Photon production is present for MT={} but no '
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=32 but no cross section is given.
  warn('Photon production is present for MT={} but no '


N,INL
N,G


/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=28 but no cross section is given.
  warn('Photon production is present for MT={} but no '
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=32 but no cross section is given.
  warn('Photon production is present for MT={} but no '
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=28 but no cross section is given.
  warn('Photon production is present for MT={} but no '
/global/scratch/users/ikolaja/conda/python3_11_6/lib/python3.11/site-packages/openmc/data/neutron.py:601: UserWarning: Photon production is present for MT=32 but no cross section is given.
  warn('Photon production is present for MT={} but no '


N,TOT
N,EL
N,INL
N,G
N,TOT
/global/scratch/users/co_nuclear/endf71x/Yb/70168.710nc not found. Evaluation does not exist?
N,EL
N,INL
N,G
/global/scratch/users/co_nuclear/endf71x/Yb/70168.710nc not found. Evaluation does not exist?
N,TOT
/global/scratch/users/co_nuclear/endf71x/Yb/70170.710nc not found. Evaluation does not exist?
N,EL
N,INL
N,G
/global/scratch/users/co_nuclear/endf71x/Yb/70170.710nc not found. Evaluation does not exist?
N,TOT
/global/scratch/users/co_nuclear/endf71x/Yb/70171.710nc not found. Evaluation does not exist?
N,EL
N,INL
N,G
/global/scratch/users/co_nuclear/endf71x/Yb/70171.710nc not found. Evaluation does not exist?
N,TOT
/global/scratch/users/co_nuclear/endf71x/Yb/70172.710nc not found. Evaluation does not exist?
N,EL
N,INL
N,G
/global/scratch/users/co_nuclear/endf71x/Yb/70172.710nc not found. Evaluation does not exist?
N,TOT
/global/scratch/users/co_nuclear/endf71x/Yb/70173.710nc not found. Evaluation does not exist?
N,EL
N,INL
N,G
/global/scratch/users/co_nuc

In [10]:
summary_list_headers = ["Z", "A", "Symbol", "Projectile", "Reaction", "MT", "has_endf7-1", "has_endf8"]
summary_df = pd.DataFrame(reaction_summary_list, columns = summary_list_headers)
display(summary_df)
file_name = output_directory + f"all_reactions.csv"
summary_df.to_csv(file_name)

,Z,A,Symbol,Projectile,Reaction,MT,has_endf7-1,has_endf8
0,1,1,H,n,1,"N,TOT",True,True
1,1,1,H,n,2,"N,EL",True,True
2,1,1,H,n,4,"N,INL",False,False
3,1,1,H,n,102,"N,G",True,True
4,1,2,H,n,1,"N,TOT",True,True
...,...,...,...,...,...,...,...,...
825,98,250,Cf,n,1,"N,TOT",True,True
826,98,250,Cf,n,102,"N,G",True,True
827,98,251,Cf,n,1,"N,TOT",True,True
828,98,252,Cf,n,102,"N,G",True,True


In [ ]:
display(summary_df)